<br>

# Introdução

In [ ]:
#!pip3 install open_geodata --upgrade

In [ ]:
import os
import re
import folium
import pandas as pd
import seaborn as sns
import geopandas as gpd
from folium import plugins

In [ ]:
from paths import *

In [ ]:
from open_geodata import lyr

<br>

## Read Data

In [ ]:
gdf = gpd.read_file(
    os.path.join(output_path_geo, 'outorgas.gpkg')
)

gdf = gdf.sample(n=3000)

gdf.head()
gdf.info()

<br>

## Layer

In [ ]:
def lyr_outorgas():
    # Input
    gdf = gpd.read_file(os.path.join(output_path_geo, 'outorgas.gpkg'))
    gdf = gdf.to_crs(epsg=4326)

    # Set dictionary
    colors = {
        'AMPLIACAO DE LEITO': '#a6cee3',
        'BARRAMENTO': '#1f78b4',
        'BATERIA DE POCOS': '#b2df8a',
        'CANALIZACAO': '#33a02c',
        'CAPT SUBTERRANEA - PORT.': '#fb9a99',
        'CAPTACAO EM NASCENTE': '#e31a1c',
        'CAPTACAO SUBTERRANEA': '#fdbf6f',
        'CAPTACAO SUPERFICIAL': '#ff7f00',
        'CONTENCAO DE MARGEM': '#cab2d6',
        'DERIVACAO': '#6a3d9a',
        'DESASSOREAMENTO': '#ffff99',
        'DIQUE': '#b15928',
        'DRENAGEM (REBAIXAMENTO)': '#a6cee3',
        'ECLUSA': '#1f78b4',
        'EMISSARIO': '#b2df8a',
        'EXTRACAO DE AGUA SUBTERRA': '#33a02c',
        'EXTRACAO DE AREIA': '#fb9a99',
        'EXTRACAO MINERIOS - CLASS': '#e31a1c',
        'FONTE ( AGUA SUPERFICIAL': '#fdbf6f',
        'LANCAMENTO EM CAVA DE MIN': '#ff7f00',
        'LANCAMENTO EM REDE': '#cab2d6',
        'LANCAMENTO EM SOLO': '#6a3d9a',
        'LANCAMENTO SUPERFICIAL': '#ffff99',
        'OUTROS': '#b15928',
        'PIER / CAIS': '#a6cee3',
        'PISCINAO': '#1f78b4',
        'POCO DE MONITORAMENTO': '#b2df8a',
        'POLDER': '#33a02c',
        'PROTECAO DE LEITO / MARGE': '#fb9a99',
        'PROTECAO DE MARGEM': '#e31a1c',
        'RECIRCULACAO': '#fdbf6f',
        'RESERVACAO (TQ SUB-SUPERF': '#ff7f00',
        'RETIFICACAO': '#cab2d6',
        'REVERSAO': '#6a3d9a',
        'TANQUE': '#ffff99',
        'TRAVESSIA': '#b15928',
        'TRAVESSIA AEREA': '#a6cee3',
        'TRAVESSIA INTERMEDIARIA': '#1f78b4',
        'TRAVESSIA SUBTERRANEA': '#b2df8a'
    }

    # Popup
    #gdf['popup'] = gdf.apply(popup_html, axis=1)

    # Delete Columns
    gdf.drop(
        [
            # 'id_municipio', 'endereco', 'numero', 'bairro',
            # 'municipio_sede', 'cep', 'telefone', 'fax',
            # 'complemento', 'url', 'email'
        ],
        axis=1,
        inplace=True
    )

    # Layer
    return folium.GeoJson(
        gdf,
        name='Outorgas',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        control=True,
        show=False,
        style_function=lambda x: {
            'fillColor': colors[x['properties']['uso']],
            'color': colors[x['properties']['uso']],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['nome_rio_aquifero', 'uso'],
            aliases=['nome_rio_aquifero', 'uso'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        # popup=folium.GeoJsonPopup(
        #     ['popup'],
        #     parse_html=False,
        #     max_width='400',
        #     show=False,
        #     labels=False,
        #     sticky=True,
        #     # localize=True,
        #     # style=f"""
        #     # background-color: #F0EFEF;
        #     # """,
        # )
    )
# Add Field


def popup_html(row):
    tel = str(row['telefone']).replace('-', '').replace(')',
                                                        '').replace('(', '+55').replace(' ', '')
    fax = str(row['fax']).replace('-', '').replace(')',
                                                   '').replace('(', '+55').replace(' ', '')

    html = """
    <div>
    <p><b>{}</b> pertence à:
    <h4><b>{}</b></h4></p>
    
    <p><b>Sede:</b><br>
    {}{}{}<br>
    {}
    {}
    {}</p>
    <p><b>Contatos:</b><br>
    {}
    {}
    {}</p>
    <p>{}</p>
    </div>
    """.format(
        '' if pd.isnull(row['municipio_nome']) else '{}'.format(
            row['municipio_nome']),
        '' if pd.isnull(row['agencia']) else '{}'.format(row['agencia']),
        '' if pd.isnull(row['endereco']) else '{}'.format(row['endereco']),
        '' if pd.isnull(row['numero']) else ', {}'.format(row['numero']),
        '' if pd.isnull(row['complemento']
                        ) else ' - {}'.format(row['complemento']),
        '' if pd.isnull(row['bairro']) else 'Bairro: {}<br>'.format(
            row['bairro']),
        '' if pd.isnull(row['municipio_sede']) else 'Município: {}<br>'.format(
            row['municipio_sede']),
        '' if pd.isnull(row['cep']) else 'CEP: {}<br>'.format(row['cep']),
        '' if pd.isnull(
            row['telefone']) else 'Telefone: <a href="tel:{}">{}</a><br>'.format(tel, row['telefone']),
        '' if pd.isnull(
            row['fax']) else 'Fax: <a href="tel:{}">{}</a><br>'.format(fax, row['fax']),
        '' if pd.isnull(
            row['email']) else 'E-mail: <a href="mailto:{}">{}</a>'.format(row['email'], row['email']),
        '' if pd.isnull(
            row['email']) else '<a href="{}" target="_blank">Conferir Dados</a>'.format(row['url']),
    )

    html = html.replace('\n', '')
    html = re.sub('\s\s+', ' ', html)  # Remove Espaços no meio
    html = html.strip()
    return html


<br>

## Folium

In [ ]:
def get_map(input_geojson):
    # Input
    gdf = gpd.read_file(input_geojson)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]
    
    # Zoom
    min_zoom = 8
    max_zoom = 21
    padding = 1
    
    # Create Map
    m = folium.Map(
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        min_lat=bounds[0][0]*((100+padding)/100),
        min_lon=bounds[0][1]*((100+padding)/100),
        max_lat=bounds[1][0]*((100-padding)/100),
        max_lon=bounds[1][1]*((100-padding)/100),
        tiles=None,
        # zoom_delta=0.1,
        # zoom_start=10,
    )
    
    # Add Layers
    m.add_child(lyr.base.google_streets(min_zoom, max_zoom))
    m.add_child(lyr.base.google_terrain(min_zoom, max_zoom))
    m.add_child(lyr.base.google_hybrid(min_zoom, max_zoom))
    m.add_child(lyr.base.google_satellite(min_zoom, max_zoom))

    # Monitoramento
    m.add_child(lyr_outorgas())

    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False,
    ).add_to(m)
    return m

In [ ]:
# Create Maps
m = get_map(
    os.path.join(output_path_geo, 'outorgas.gpkg')
)

# Save
#m.save(os.path.join(output_path_map, 'outorgas_map.html'))
m